In [1]:
import pandas as pd
import numpy as np
import os
pd.options.display.float_format = '{:,.2f}'.format
import duckdb
!pip install xlrd openpyxl 
con = duckdb.connect(database=':memory:')


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
games = pd.read_table("games.csv",delimiter=",",header=0,
                      usecols=["gameID","leagueID","season","homeTeamID","awayTeamID","homeGoals","awayGoals"])
games.info()
games.head()

teams = pd.read_excel("UEFA5.xlsx",sheet_name="teams",header=0)
teams.info()
teams.head()

leagues = pd.read_excel("UEFA5.xlsx",sheet_name="leagues",header=0)
leagues.info()
leagues.head()

players = pd.read_excel("UEFA5.xlsx",sheet_name="players",header=0)
players.info()
players.head()

homes = (games
         .eval('''teamID=homeTeamID
                  GolesAF=homeGoals
                  GolesEC=awayGoals''')
         .loc[:,["gameID","leagueID","season","teamID","GolesAF","GolesEC"]]
        )

away = (games
        .eval('''teamID=awayTeamID
                 GolesAF=awayGoals
                 GolesEC=homeGoals''')
         .loc[:,["gameID","leagueID","season","teamID","GolesAF","GolesEC"]]
        )

todo = pd.concat([homes,away], ignore_index=True)
todo.loc[todo["GolesAF"]  < todo["GolesEC"],"Puntos"] = 0
todo.loc[todo["GolesAF"]  > todo["GolesEC"],"Puntos"] = 3
todo.loc[todo["GolesAF"] == todo["GolesEC"],"Puntos"] = 1
todo.info()
todo.head()

todo2 = (todo
         .merge(teams,left_on="teamID",right_on="teamID",how="left")
         .merge(leagues,left_on="leagueID",right_on="leagueID",how="left")
        ) 
todo2.info()
todo2.head()

appearances = pd.read_table("appearances.csv",delimiter=",",header=0,quotechar="'",usecols=[0,1,2,15],
                            names=["gameID","playerID","goals","time"]
                           )
appearances["playerID"] = pd.to_numeric(appearances["playerID"].str.replace('"',"",regex=True))
appearances["gameID"] = pd.to_numeric(appearances["gameID"].str.replace('"',"",regex=True))
appearances.info()
appearances.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12680 entries, 0 to 12679
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   gameID      12680 non-null  int64
 1   leagueID    12680 non-null  int64
 2   season      12680 non-null  int64
 3   homeTeamID  12680 non-null  int64
 4   awayTeamID  12680 non-null  int64
 5   homeGoals   12680 non-null  int64
 6   awayGoals   12680 non-null  int64
dtypes: int64(7)
memory usage: 693.6 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   teamID    146 non-null    int64 
 1   teamNAME  146 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   leagueID  

,gameID,playerID,goals,time
0,81,560,0,90
1,81,557,0,82
2,81,548,0,90
3,81,628,0,90
4,81,1006,0,90


In [3]:
#P1
resultados = (todo2
              .groupby(["leagueID","leagueNAME","season","teamID","teamNAME"])
              .agg(puntos=("Puntos","sum"),golesAF=("GolesAF","sum"),golesEC=("GolesEC","sum")).reset_index()
              .eval("golDIF=golesAF-golesEC")
	  		  .drop(columns=["golesAF","golesEC"])
              .sort_values(by=["leagueID","leagueNAME","season","puntos","golDIF"],
                           ascending=[True,True,True,False,False]).reset_index(drop=True)
             )

resultados2 = con.execute("""select   leagueID, leagueNAME, season, teamID, teamNAME, sum(Puntos) as puntos,
                                                                                      sum(golesAF)-sum(golesEC) as golDIF 
			                 from     todo2
			                 group by leagueID, leagueNAME, season, teamID, teamNAME
			                 order by leagueID, leagueNAME, season, puntos desc, golDIF desc
			                 limit    2""").df()


resultados.query("season==2016 and leagueNAME=='Premier League'").head(2)

con.execute("""select   *
               from     resultados2
			   where 	season=2016 and leagueNAME='Premier League'
               limit    2""").df()

#80-Chelsea 93 52

,leagueID,leagueNAME,season,teamID,teamNAME,puntos,golDIF


In [4]:
#P2
resultados.query("season==2017 and leagueNAME=='Serie A'").head(3)

con.execute("""select   *
               from     resultados2
			   where 	season=2017 and leagueNAME='Serie A'
               limit    3""").df()

#105-Juventus 91	48

,leagueID,leagueNAME,season,teamID,teamNAME,puntos,golDIF


In [5]:
#P3
resultados.query("season==2018 and leagueNAME=='Bundesliga'").head(4)

con.execute("""select   *
               from     resultados2
			   where 	season=2018 and leagueNAME='Bundesliga'
               limit    4""").df()

#136-Bayern Munich 66 34	

,leagueID,leagueNAME,season,teamID,teamNAME,puntos,golDIF


In [6]:
#P4
resultados.query("season==2019 and leagueNAME=='La Liga'").head(5)

con.execute("""select   *
               from     resultados2
			   where 	season=2019 and leagueNAME='La Liga'
               limit    5""").df()

#138-Real Madrid 70 20

,leagueID,leagueNAME,season,teamID,teamNAME,puntos,golDIF


In [7]:
#P5
resultados.query("season==2020 and leagueNAME=='Ligue 1'").head(6)

con.execute("""select   *
               from     resultados2
			   where 	season=2020 and leagueNAME='Ligue 1'
               limit    6""").df()

#164-Lille 60 7

,leagueID,leagueNAME,season,teamID,teamNAME,puntos,golDIF


In [8]:
#P6
temp = todo2[["gameID","leagueID","season","leagueNAME"]].drop_duplicates()
appearances2 = (temp
                .merge(appearances,left_on=["gameID"],right_on=["gameID"],how="inner")
                .merge(players,left_on=["playerID"],right_on=["playerID"],how="inner")
               )
goles = (appearances2
         .groupby(["leagueID","leagueNAME","season","playerID","playerNAME"])
         .agg(goles=("goals","sum"),tiempo=("time","sum")).reset_index()
         .eval('''golesxtiempo = 90*goles/tiempo''')
         .query("goles > 10")
		 .drop(columns=["goals","tiempo"])
         .sort_values(by=["leagueID","leagueNAME","season","golesxtiempo"],
                      ascending=[True,True,True,False]).reset_index(drop=True)
        )

goles2 = con.execute("""select   leagueID, leagueNAME, season, playerID, playerNAME, 90*sum(goals)/sum(time) as golesxtiempo
			            from     appearances2
			            group by leagueID, leagueNAME, season, playerID, playerNAME
						having 	 sum(goals) > 10
			            order by leagueID, leagueNAME, season, golesxtiempo desc
			            limit    2""").df()



goles.query("season==2016 and leagueNAME=='Ligue 1'").head(2)

con.execute("""select   *
               from     goles2
			   where 	season=2016 and leagueNAME='Ligue1'
               limit    2""").df()

#3294-Edinson Cavani 1.0560

KeyError: "['goals'] not found in axis"

In [0]:
#P7
goles.query("season==2017 and leagueNAME=='La Liga'").head(2)

con.execute("""select   *
               from     goles2
			   where 	season=2017 and leagueNAME='La Liga'
               limit    2""").df()

#2097-Lionel Messi 1.0217

In [ ]:
#P8
goles.query("season==2018 and leagueNAME=='Bundesliga'").head(2)

con.execute("""select   *
               from     goles2
			   where 	season=2018 and leagueNAME='Bundesliga'
               limit    2""").df()

#2380-Paco Alcácer 1.3717

In [ ]:
#P9
goles.query("season==2019 and leagueNAME=='Serie A'").head(2)

con.execute("""select   *
               from     goles2
			   where 	season=2019 and leagueNAME='Serie A'
               limit    2""").df()

#1229-Luis Muriel 1.3224	

In [ ]:
#P10
goles.query("season==2020 and leagueNAME=='Premier League'").head(2)

con.execute("""select   *
               from     goles2
			   where 	season=2020 and leagueNAME='Premier League'
               limit    2""").df()
			   
#2251-Gareth Bale 1.0891	

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cd2fca3f-dc64-4c71-8bd4-adda531fa911' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>